In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import os
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, classification_report
import onnxruntime as ort

### Model size

In [43]:
model_qat_size = os.path.getsize('model_qat.onnx')/(1024**2)
model_ptq_size = os.path.getsize('model_ptq.onnx')/(1024**2)

In [44]:
print(f"Model QAT size: {model_qat_size:.2f} MB")
print(f"Model PTQ size: {model_ptq_size:.2f} MB")

Model QAT size: 0.42 MB
Model PTQ size: 0.42 MB


### Performance metrics

In [45]:
def dataset_creator(path_to_folder, max_by_experiment = 3):
    fire = 0
    images_ptq, images_qat, labels = [], [], []


    for folder in os.listdir(path_to_folder):

        if os.path.isdir(os.path.join(path_to_folder, folder)):

            for file in os.listdir(os.path.join(path_to_folder, folder)):

                if file.endswith('.txt'):

                    if 'fire' in file and fire < max_by_experiment:
                        fire += 1
                        for pic in sorted(os.listdir(os.path.join(path_to_folder, folder, 'images'))):
                            img = load_img(os.path.join(os.path.join(path_to_folder, folder, 'images'), pic), target_size=(200, 200), color_mode='grayscale')  # Ajustar tamaño y modo de color
                            img_array = img_to_array(img) / 128.0  -1 # Normalizar la imagen
                            images_ptq.append(img_array)
                            img_array = img_to_array(img) / 255.0
                            images_qat.append(img_array)

                        
                        labels_txt = np.loadtxt(os.path.join(path_to_folder, folder, file), delimiter=' ')

                        for label in labels_txt:
                            label = [np.array([np.nan]*2), label, np.array([np.nan])]
                            label = pad_sequences(label, dtype='float32', padding='post', value=np.nan)
                            labels.append(label)



    return np.array(images_ptq), np.array(images_qat), np.array(labels)

images_test_ptq, images_test_qat, labels_test = dataset_creator('../../testing', 20)
indices = np.random.permutation(images_test_ptq.shape[0])
images_test_ptq = images_test_ptq[indices]
images_test_qat = images_test_qat[indices]
labels_test = labels_test[indices]
_, y_fire_test, _ = labels_test[:,0, :][:, :2], labels_test[:, 1, :], labels_test[:, 2, :][:, 0]

images_test_ptq = images_test_ptq.reshape(images_test_ptq.shape[0], 1, 200, 200)
images_test_qat = images_test_qat.reshape(images_test_qat.shape[0], 1, 200, 200)

In [46]:
model_qat = ort.InferenceSession('model_qat.onnx')
model_ptq = ort.InferenceSession('model_ptq.onnx')

In [47]:
outs_qat = [out.name for out in model_qat.get_outputs()]
outs_ptq = [out.name for out in model_ptq.get_outputs()]

In [48]:
predictions_qat = []
predictions_ptq = []

for i in range(images_test_ptq.shape[0]):
    input_data_ptq = [images_test_ptq[i]]
    input_data_qat = [images_test_qat[i]]
    pred_qat = model_qat.run(outs_qat, {'input': input_data_qat})
    pred_ptq = model_ptq.run(outs_ptq, {'input': input_data_ptq})
    predictions_qat.append(pred_qat)
    predictions_ptq.append(pred_ptq)

predictions_qat_ee = np.squeeze(np.argmax(predictions_qat, axis=-1), axis=-1)[:, 0]
predictions_qat_final = np.squeeze(np.argmax(predictions_qat, axis=-1), axis=-1)[:, 1]
predictions_ptq_ee = np.squeeze(np.argmax(predictions_ptq, axis=-1), axis=-1)[:, 0]
predictions_ptq_final = np.squeeze(np.argmax(predictions_ptq, axis=-1), axis=-1)[:, 1]

In [49]:
labels_class = np.argmax(y_fire_test, axis=-1)

acc_qat_ee = accuracy_score(labels_class, predictions_qat_ee)
acc_ptq_ee = accuracy_score(labels_class, predictions_ptq_ee)

acc_qat = accuracy_score(labels_class, predictions_qat_final)
acc_ptq = accuracy_score(labels_class, predictions_ptq_final)

print(f"Accuracy QAT early exit: {acc_qat_ee:.2f}")
print(f"Accuracy PTQ early exit: {acc_ptq_ee:.2f}")

print(f"Accuracy QAT: {acc_qat:.2f}")
print(f"Accuracy PTQ: {acc_ptq:.2f}")

Accuracy QAT early exit: 0.70
Accuracy PTQ early exit: 0.74
Accuracy QAT: 0.69
Accuracy PTQ: 0.76


In [53]:
pd.DataFrame({'QAT': [acc_qat_ee, acc_qat], 'PTQ': [acc_ptq_ee, acc_ptq]}, index=['Early Exit', 'Final']).to_csv('performance_table.csv')

In [50]:
print("Classification report QAT, early exit:\n")
print(classification_report(labels_class, predictions_qat_ee), '\n\n')

print("Classification report QAT:\n")
print(classification_report(labels_class, predictions_qat_final), '\n\n')

print("Classification report PTQ, early exit:\n")
print(classification_report(labels_class, predictions_ptq_ee), '\n\n')

print("Classification report PTQ:\n")
print(classification_report(labels_class, predictions_ptq_final), '\n\n')

Classification report QAT, early exit:

              precision    recall  f1-score   support

           0       0.49      0.52      0.50      2127
           1       0.58      0.73      0.65      1278
           2       0.82      0.69      0.75      4059
           3       0.79      0.86      0.82      2420

    accuracy                           0.70      9884
   macro avg       0.67      0.70      0.68      9884
weighted avg       0.71      0.70      0.70      9884
 


Classification report QAT:

              precision    recall  f1-score   support

           0       0.49      0.51      0.50      2127
           1       0.52      0.69      0.59      1278
           2       0.81      0.69      0.74      4059
           3       0.80      0.84      0.82      2420

    accuracy                           0.69      9884
   macro avg       0.66      0.68      0.66      9884
weighted avg       0.70      0.69      0.69      9884
 


Classification report PTQ, early exit:

              pr